In [ ]:
import pandas as pd
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt


#import the file
file=pd.read_excel(r'/Users/natalieneri/SeniorSpring/Python/italy data.xlsx')
df=pd.DataFrame(file,columns=['Days Since Dec 30','Total Cases'])

#convert the data to a matrix, then make the total cases into a list
data=df.as_matrix()
cases=data[30:108,1].tolist()

#create a vector for the days since jan 30 (day before first case)
numDays=len(cases)
dayNum=np.arange(numDays)

plt.plot(dayNum,cases,'.',label='Actual Data',color='black')
plt.title('Total Cases In Italy From 1/30 to 4/16')
plt.xlabel('Days Since 1/30')
plt.ylabel('Total Cases')


#Find a part that looks like a rising exponential
rising=cases[0:45]
daysRising=np.arange(len(rising))

#Define a function that the system might follow
def casefunction(x,a,b,c):
    return (a*np.exp(x*b)+c)

popt,pcov= scipy.optimize.curve_fit(casefunction, daysRising, rising)


#plot the function over the data
plt.plot(daysRising,casefunction(daysRising,*popt),color='orange',label='Model Based on the Rising Exponential')
plt.legend()
plt.show()

#make another plot showing the prediction
plt.plot(dayNum,cases,'.',label='Actual Data',color='black')
plt.title('Total Cases In Italy (With Prediction)')
plt.xlabel('Days Since 1/30')
plt.ylabel('Total Cases')

plt.plot(dayNum,casefunction(dayNum,*popt),color='orange',label='Model Based on the Rising Exponential')
plt.legend()
plt.show()



In [19]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt


#ASSUMING: DEATH RATE=BIRTH RATE, 1% of COVID PATIENTS WILL NEED VENTILATION, 62,000 VENTILATORS IN US

#no mitigation techniques
def rhs1(w,t,p):
    S,E,I,R,H=w
    Ro,Tinf,Tinc=p
    
    f1=[(-Ro/Tinf*I*S)/pop, 
        (Ro/Tinf*I*S/pop-E/Tinc),
        (E/Tinc-I/Tinf),
        (I/Tinf),
       (E/Tinc-I/Tinf)*v]
        
    return f1

#mitigation techniques
def rhs(w,t,p):
    S,E,I,R,H=w
    Ro,Rt,Tinf,Tinc=p
    
    if t<=t_intervention:
        f=[(-Ro/Tinf*I*S)/pop, 
        (Ro/Tinf*I*S/pop-E/Tinc),
        (E/Tinc-I/Tinf),
        (I/Tinf),
        (E/Tinc-I/Tinf)*0.01]
    else: 
        f=[(-Rt/Tinf*I*S)/pop, 
        (Rt/Tinf*I*S/pop-E/Tinc),
        (E/Tinc-I/Tinf),
        (I/Tinf),
        (E/Tinc-I/Tinf)*v]
   
    return f


#known values that we can also change
pop=int(input('Population: '))
Ro=float(input('How many people will one infected person infect? ')) #how many people will one infected person infect
Tinf=float(input('How many days is an infected person contagious for? ')) #number of days an infected person is contagious
Tinc=float(input('What is the incubation period? '))
t_intervention=int(input('Input the day that intervention began: '))
reduction=float(input('Input the ratio by which social contact was decreased: '))
Rt=Ro-reduction*Ro #how many people one person will infect during social distancing
v=float(input('What ratio of infected people will require a ventilator? '))
ventilator_supply=int(input('Input the number of ventilators available in the area: '))
#initial conditions that we can also change
I=1 #initial number of people infected
S=pop #susceptible people. 
E=0 #number of people exposed
R=0 #Recovered
H=0 #hospitalized

#Other Parameters
starttime=0
stoptime=200 #days since number of infections =I_initial
n=stoptime+1
t=np.linspace(starttime, stoptime, n)

#pack up parameters
p=[Ro,Rt,Tinf,Tinc]
p1=[Ro,Tinf,Tinc]
w0=[S,E,I,R,H]

#solve
wsol=odeint(rhs,w0,t,args=(p,))
wsol1=odeint(rhs1,w0,t,args=(p1,))

#plot with intervention
plt.plot(t,wsol[:,2],label='60% Reduction in Social Contact')

#plot without intervention
plt.plot(t,wsol1[:,2],label='No Social Distancing',color='orange')
plt.title('Number of Infections')
plt.xlabel('Days Since First US Case')
plt.ylabel('Number of Infected People')
plt.legend(loc=1)
plt.show()

#close up of ventilators and need
plt.plot(t,wsol1[:,4],label='No Social Distancing',color='orange')
plt.plot(t,np.repeat(ventilator_supply,len(t)),'--',label='Available Ventilators',color='green')
plt.plot(t,wsol[:,4],label='60% Reduction in Social Contact')
plt.title('US Ventilator Demand')
plt.xlabel('Days Since First US Case')
plt.ylabel('Ventilator Demand')
plt.legend(loc=1)
plt.show()


Population: 7000000
How many people will one infected person infect? u


ValueError: could not convert string to float: 'u'

In [20]:
#part three


In [8]:
import pygame
import sys
import numpy as np
import random
from random import randrange 

screen_size= width,height = (700,700)
black = (0, 0, 0)
white = (255, 255, 255)
red = (255, 50, 50)
blue= (135,206,235)
green= (0,255,127)
radius = 6
N=100 #number of dots
odds=float(input('What are the odds of contracting the infection if an uninfected person comes in contact with an infected one? '))

#Initialization
pygame.init()
screen = pygame.display.set_mode(screen_size)
pygame.display.set_caption("Pandemic Simulation")

paused = False

#set initial positions and velocities
def getInitialInfo():
    xposition=random.randrange(0,700)
    yposition=random.randrange(0,700)
    
    pos=[xposition,yposition]

    angle=random.uniform(0,2*np.pi)
    xvelocity=3*np.cos(angle)
    yvelocity=3*np.sin(angle)
    vel=[xvelocity,yvelocity]
    
    return pos,vel

#initialize an empty vector to store position and velocity information
uninfected=[]
infected=[]
newinfected=[]

#Create initial uninfected dots
for i in range(N):
    #store info for the original uninfected dots
    pos,vel=getInitialInfo()
    name=i+1
    uninfected.append([pos,vel,name])

#create the first infected dot
p,v=getInitialInfo()
name1=N+1
infected.append([p,v,name1])


#use the pygame clock
clock=pygame.time.Clock()
#initialize count
counter=0

#loop continuously if the simulation is not paused
done=False


while not done: 
    for event in pygame.event.get():   
        if event.type == pygame.QUIT:  
            done = True   
 
    # Set the screen background as white
    screen.fill(white)
    
    #uninfected dots
    for i in range(len(uninfected)):
        #draw a dot
        pygame.draw.circle(screen,blue,uninfected[i][0],radius)
    
        #update the position of the dot
        uninfected[i][0][0]+=int(np.round(uninfected[i][1][0]))
        uninfected[i][0][1]+=int(np.round(uninfected[i][1][1]))
        
        #bounce the dot off the walls
        if uninfected[i][0][0]>=700 or uninfected[i][0][0]<=0:
            uninfected[i][1][0]*=-1
        if uninfected[i][0][1]>=700 or uninfected[i][0][1]<=0:
            uninfected[i][1][1]*=-1
        
        #Check if the uninfected dot hit an infected dot and make it become infected
        for j in range(len(infected)):
            xdist=abs(uninfected[i][0][0]-infected[j][0][0])
            ydist=abs(uninfected[i][0][1]-infected[j][0][1])
            #come up with a random number between 0 and 1 to determine if the person gets infected or not
            randomnum=random.uniform(0,1)
            if xdist<=2*radius and ydist<=2*radius and randomnum<=odds:
                newinfected.append(uninfected[i])
                
    #remove any of the new infected dots from the list of uninfected dots            
    uninfected=[elem for elem in uninfected if elem not in newinfected] 
    #add any new infections to the infected list
    infected.extend(newinfected)
    #empty out the new infected vector to prepare for more iterations
    newinfected.clear()
                
    #do the infected dots        
    for i in range(len(infected)):
        pygame.draw.circle(screen,red,infected[i][0],radius)
        
        #update the position of the dot
        infected[i][0][0]+=int(np.round(infected[i][1][0]))
        infected[i][0][1]+=int(np.round(infected[i][1][1]))
        
        #bounce the dots off the walls
        if infected[i][0][0]>=700 or infected[i][0][0]<=0:
            infected[i][1][0]*=-1
        if infected[i][0][1]>=700 or infected[i][0][1]<=0:
            infected[i][1][1]*=-1
        
    #display it
    pygame.display.flip()
    clock.tick(10)
    counter+=1
    #if there are no more uninfected people left, we have to show how long it took to get there
    if not uninfected:
        print('It took',counter,'loops to infect everyone with a',odds,'chance of infection')
        break
pygame.quit()




What are the odds of contracting the infection if an uninfected person comes in contact with an infected one? 1
It took 467 loops to infect everyone with a 1.0 chance of infection
